In [448]:
import matplotlib
import matplotlib.pyplot as mp
import numpy as np
import xarray as xr

##### SCAM FORCINGS GENERATION FOR SAS PARAMETERS IN #####
### http://www2.mmm.ucar.edu/people/patton/documents/su_et_al.ACP.2016.pdf ###

## Time ##

spd = 86400 # Seconds per day
dtime = 30*60    # Time interval (minutes*60=sec) for each time point on file
tperiod = (16.5-6)*spd    # Time length (hours*86400=sec) for whole of IOP
iop_zstart = 12*3600  # 12Z = 6AM MST (US)

iop_lat = 32.5 # Lat location (SAS average of two smapling sites)
iop_lon = -87.15 # Lon

vdesc = ('Initial BL height', \
         'Subsidence rate', \
         'Surface sensible heat flux',\
         'Surface latent heat flux',\
         'Entrainment/surface heat flux ratio'\
         'Initial BL potential temperature',\
         'Initial FT potential temperature',\
         'Potential temperature lapse rate FT',\
         'Advection of potential temperature',\
         'Initial BL specific humidity',\
         'Initial FT specific humidity',\
         'Specific humidity lapse rate FT',\
         'Advection of specific humidity')


vname = ('pblh','w_sub','shflx','lhflx','eratio','the_bl','the_trop','the_lr','the_adv','q_bl','q_trop','q_lr','q_adv')
vval =  (500,    9.e-6,   0.1,   0.15,    0.2,     296.6,   298.1,     0.003,   6.40e-4, 16.8,    12.8,   -0.004,   1.5e-4)

##############################


##### IOP file ######
iop_file_in = './ARM95_4scam.nc'   # input template
iop_file_out = './SAS_ideal_4scam.nc' # Output forcing file

## Read in and IOP template
iop_in = xr.open_dataset(iop_file_in,engine='netcdf4') # This is like addfile -> pointer

### Copy IOP 'DataSet' ##
iop_out = iop_in
print(iop_in.vertdivq )
## Clean out existing variables ##
iop_out = iop_in.drop(iop_in.data_vars)

## Fill out known info. now ##
iop_out['lon'] = [iop_lon] # need [] here so it gets treated as a 0d array coordinate
iop_out['lat'] = [iop_lat]

## Add Time (tsec) Array Based on time information ##
ntsteps = int(tperiod/dtime)
time = iop_zstart+dtime*np.arange(ntsteps)

## Copy attributes of tsec from iop_in ##
iop_out.coords['tsec'] = ('time', time)
iop_out.tsec.attrs['long_name'] = iop_in.tsec.attrs['long_name'] 
iop_out.tsec.attrs['units'] = iop_in.tsec.attrs['units'] 

## Open New IOP/netcdf/DataSet ##
iop_out.to_netcdf(iop_file_out)

## Set up a base 3D/4D DataArray that will be written into the new DataSet
var_4d = np.zeros((ntsteps,18,1,1)) # Set-up
var_3d = np.zeros((ntsteps,1,1)) # Set-up

iop_out_c4 = ('time','lev','lat','lon')   # Coordinates for iop_out
iop_out_c3 = ('time','lat','lon')



                 

<xarray.DataArray 'vertdivq' (time: 1261, lev: 18, lat: 1, lon: 1)>
array([[[[ 1.537382e-11]],

        ...,

        [[-4.665427e-09]]],


       ...,


       [[[ 3.091134e-13]],

        ...,

        [[ 3.121381e-10]]]], dtype=float32)
Coordinates:
  * lon      (lon) float32 -97.49
  * lat      (lat) float32 36.61
  * lev      (lev) float32 11500.0 16500.0 21500.0 ... 86500.0 91500.0 96500.0
Dimensions without coordinates: time
Attributes:
    long_name:  Vertical Q Advective Tendency
    units:      kg/kg/s


In [444]:
### Need constants ###

r_gas = 287.   # Specific gas constant for dry air
cp_air = 1004. # Specific heat for dry air

r_cp = r_gas/cp_air    # r/cp
grav = 9.81     # Gravity ave.

                        
###### Construct time varying data for 3d arrays (time,lat,lon) ######

## Generate boundary forced variables ##
shflx0 = vval[vname.index('shflx')]
vdesc_i = vdesc[vname.index('shflx')]
lhflx0 = vval[vname.index('lhflx')]
vdesc_i = vdesc[vname.index('lhflx')]

## SAS case study: Daylight variation of sfce fluxes ##
shflx = np.copy(var_3d)  
shflx[:,0,0] = shflx0*np.sin(np.pi*(time-iop_zstart)/tperiod)
iop_out['shflx'] = (iop_out_c3, var_3d)

lhflx = np.copy(var_3d)  
lhflx[:,0,0] = lhflx0*np.sin(np.pi*(time-iop_zstart)/tperiod)
iop_out['lhflx'] = (iop_out_c3, lhflx)


In [445]:
### Constructing vertical profiles ###
## Read in SCAM IOP template here (split into a function at some point)
#plevs = np.array([10, 20, 100, 150, 200, 300, 400, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 975, 1000])
#plevs = np.arange(10,1000,10)


## Convert trop theta values into temperature given plevs ##
plevs = iop_in['lev']

nplevs = np.size(plevs)
ps = 101500. # psurf [pa]
p0 = 100000. # pref [pa]

## Z of plevels, tricky don't have T yet ##
dp_levs = np.diff(p)


##### Construct profile ICs #######

## 1. Grab CASE values ##
pblh =  vval[vname.index('pblh')]

the_ft = vval[vname.index('the_trop')]
the_bl = vval[vname.index('the_bl')]
dthedp_ft =  vval[vname.index('the_lr')]

q_ft = vval[vname.index('q_trop')]
q_bl = vval[vname.index('q_bl')]
dqdp_ft =  vval[vname.index('q_lr')]



## 2. Find PBL top level ##

# Estimate z levels from p levels.
z_plevs = (r_gas/grav)*t_pbl*np.log(ps/plevs)

# Where's the PBL?
ipbl_levs = np.where(z_plevs <= pblh)
npbl_levs = np.size(ipbl_levs)
nft_levs = nplevs-npbl_levs

ipbl_min = np.amin(ipbl_levs)


## 3. Use Gradient+mean and specified theta, q for profiles ##

q_plevs = np.full(nplevs, q_bl)
the = np.full(nplevs, the_bl) # Initialize theta pbl as numpys

# Construct profiles in the FT, starting at ipbl+ and working up

for ip in range(ipbl_min-1,-1,-1):
    print("LEV --> ",ip,plevs[ip].data)
    
    # Temp profile
    t_ip = the[ip-1]*(plevs[ip-1]/p0)**r_cp # Temp at the level below
    rho = plevs[ip]/(r_gas*t_ip)  # Density at lev-=ip
    dz = dp_levs[ip]/(rho*grav)
    the[ip] = the[ip+1]+dthedp_ft*dz

    print("T ->",t_ip.data,rho.data,dz.data,the[ip])
    
    # q profile 
 
    q_plevs[ip] = q_plevs[ip+1]+dqdp_ft*dz

    print("q ->",q_plevs[ip],dz.data,dqdp_ft*dz.data)
    print("")

## Set temp
temp_plevs = the*(plevs/p0)**r_cp

## Don't let q go below zero
q_plevs[np.where(q <= 0)] = 0.1
    
### Set height invarient quantities ##    

    
## Quick plots ##

fig, axs = plt.subplots(2)
fig.suptitle('T/q')
axs[0].plot(temp_plevs, plevs[::-1])
axs[1].plot(q, plevs[::-1])

#z_plevs = (r_gas/grav)*np.mean(temp_plevs)*np.log(ps/plevs)



LEV -->  16 91500.0
T -> 284.5553707491009 1.1203982204628984 454.91325014060567 297.9647397504218
q -> 14.980346999437579 454.91325014060567 -1.8196530005624227

LEV -->  15 86500.0
T -> 279.7531369719497 1.077356027121929 473.08780300241915 299.3840031594291
q -> 13.087995787427902 473.08780300241915 -1.8923512120096766

LEV -->  14 81500.0
T -> 274.7356408582365 1.033619535304739 493.10600130275145 300.86332116333733
q -> 11.115571782216897 493.10600130275145 -1.9724240052110058

LEV -->  13 76500.0
T -> 269.4781585536641 0.9891359065191271 515.2820684835509 302.40916736878796
q -> 9.054443508282693 515.2820684835509 -2.061128273934204

LEV -->  12 71500.0
T -> 263.95114439616793 0.9438448180649148 540.0082578908364 304.0291921424605
q -> 6.894410476719347 540.0082578908364 -2.1600330315633456

LEV -->  11 66500.0
T -> 258.11884354666375 0.8976768758507232 567.7811355444613 305.7325355490939
q -> 4.623285934541502 567.7811355444613 -2.271124542177845

LEV -->  10 61500.0
T -> 251.93

IndexError: too many indices for array

In [ ]:
## Write out data to 4d array, just copying initial temp to all times ##

t = np.copy(var_4d)  
for it in range(0, ntsteps-1):
   t[it,:,0,0] = temp_plevs.data[:]
iop_out['T'] = (iop_out_c4, t)

q = np.copy(var_4d) 
for it in range(0, ntsteps-1):
   q[it,:,0,0] = q_plevs.data[:]
iop_out['Q'] = (iop_out_c4, q)

q = np.copy(var_4d) 
for it in range(0, ntsteps-1):
   q[it,:,0,0] = q_plevs.data[:]
iop_out['Q'] = (iop_out_c4, q)


iop_out